In [ ]:
import os
file = '/kaggle/input/utkface-new/crop_part1/'
folderfile = os.listdir(file)
print(folderfile[0:5])
print(len(folderfile))

In [ ]:
csvfile = []
for i in range(len(folderfile)):
    if folderfile[i][1]=='_':    #0~9歳のファイル
        label = folderfile[i][0]
    elif folderfile[i][2]=='_':    #10~99歳のファイル
        label = folderfile[i][0:2]
    else:                       #100歳以上のファイル
        label = folderfile[i][0:3]
        
    path = folderfile[i]
    path = '/kaggle/input/utkface-new/crop_part1/' + path
    listi = [path,label]
    csvfile.append(listi)

In [ ]:
os.makedirs('./csv')

In [ ]:
import csv
with open('./csv/age1.csv', 'w') as f:
 
    writer = csv.writer(f)
#    writer.writerow(header)
    writer.writerows(csvfile)

f.close()
#print(body)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL
import csv
import torch
import torchvision

In [ ]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, label_path, transform=None):
        x = []
        y = []
        file = open(label_path, 'r')
        data = csv.reader(file)
        for row in data:
            x.append(row[0])
            y.append(float(row[1]))
        file.close()
    
        self.x = x    
        self.y = torch.from_numpy(np.array(y)).float().view(-1, 1)
     
        self.transform = transform
  
  
    def __len__(self):
        return len(self.x)
  
  
    def __getitem__(self, i):
        img = PIL.Image.open(self.x[i]).convert('RGB')
        if self.transform is not None:
              img = self.transform(img)
    
        return img, self.y[i]

In [ ]:
transform = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Resize((26,26)),
#      torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

data_dir = './csv/age1.csv'

dataset = MyDataset(data_dir, transform=transform)

# train_data_dir = './csv/effspin5_train.csv'



# trainset = MyDataset(train_data_dir, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)


In [ ]:
import numpy as np
import torch
import torchvision.datasets as datasets
import torch.nn as nn


# 学習データ、検証データに 8:2 の割合で分割する。
# train_size = int(0.7 * len(dataset))
indices = np.arange(len(dataset))

train_dataset = torch.utils.data.Subset(dataset, indices[0:8000])
val_dataset = torch.utils.data.Subset(dataset, indices[8000:9500])
test_dataset = torch.utils.data.Subset(dataset, indices[9500:])

print(f"full: {len(dataset)} -> train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
validloader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=False)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
class RegressionNet(torch.nn.Module):

    def __init__(self):
        super(RegressionNet, self).__init__()

        self.conv1 = torch.nn.Conv2d(3, 16, 3)
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(16, 32, 3)
        self.pool2 = torch.nn.MaxPool2d(2, 2)

        self.fc1 = torch.nn.Linear(32 *5 * 5, 1024)
        self.fc2 = torch.nn.Linear(1024, 1024)
        self.fc3 = torch.nn.Linear(1024, 1)


    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.nn.functional.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = x.view(-1, 32 * 5 * 5)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

net = RegressionNet()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)
print(device)

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.L1Loss()

In [ ]:
import time
num_epocs = 10

list1=[]
list2=[]

train_loss = []
valid_loss = []
time_sta = time.time()
for epoch in range(num_epocs):
  # 学習
    net.train()
    running_train_loss = 0.0
    running_acc = 0.0
    with torch.set_grad_enabled(True):
        for data in trainloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            a = outputs.tolist()
            b = labels.tolist()
            list1.append(a)
            list2.append(b)
            loss = criterion(outputs, labels)
            running_train_loss += loss.item()
            loss.backward()
            optimizer.step()

    train_loss.append(running_train_loss / len(trainloader))
  
  # 検証
    net.eval()
    running_valid_loss = 0.0
    val_running_acc = 0.0
    with torch.set_grad_enabled(False):
        for data in validloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_valid_loss += loss.item()

    valid_loss.append(running_valid_loss / len(validloader))

    print('#epoch:{}\ttrain loss: {}\tvalid loss: {}'.format(epoch,
                                                running_train_loss / len(trainloader), 
                                                running_valid_loss / len(validloader)))
    
time_end = time.time()
tim = time_end- time_sta
print(tim)

In [ ]:
pip install japanize_matplotlib

In [ ]:
import japanize_matplotlib
plt.plot(train_loss, label='学習')
plt.plot(valid_loss, label='検証')
plt.xlabel('繰り返し回数',fontsize=14)
plt.ylabel('損失',fontsize=14)
plt.title('学習曲線(損失)',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)
plt.show()

In [ ]:
test_loss=[]
running_test_loss = 0.0
pred=[]
ans=[]
with torch.set_grad_enabled(False):
    for data in testloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        list1 = labels.tolist()
        outputs = net(inputs)
        list2 = outputs.tolist()
        for i in range(len(list1)):
            ans.append(list1[i])
        for i in range(len(list2)):
            pred.append(list2[i])
        loss = criterion(outputs, labels)
        running_test_loss += loss.item()

test_loss.append(running_test_loss / len(testloader))

print('test loss: {}'.format(running_test_loss / len(testloader)))
                

In [ ]:
import collections

def flatten(l):
    for el in l:
        if isinstance(el, collections.abc.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el
            

In [ ]:
ans= list(flatten(ans))
pred= list(flatten(pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
warnings.simplefilter('ignore')
import japanize_matplotlib
import statistics
ans= list(flatten(ans))
pred= list(flatten(pred))

abser=[]
for i in range(len(ans)):
    abser.append(abs(pred[i]-ans[i]))
    
plt.hist(abser,bins=80)
plt.xlabel('絶対誤差',fontsize=16)
plt.ylabel('度数',fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

print(max(abser))
print(min(abser))
print(statistics.mean(abser))
print(statistics.median(abser))
print(statistics.stdev(abser))

In [ ]:
from PIL import Image

x = []
y = []
file = open('./csv/age1.csv', 'r')
data = csv.reader(file)
for row in data:
    x.append(row[0])
    y.append(float(row[1]))
file.close()

In [ ]:
imagelist=[]
labellist=[]
for i in range(40):
    imagelist.append(Image.open(x[9500+i]))
    labellist.append(ans)
    # plt.imshow(Image.open(x[i]))

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot  as plt
import japanize_matplotlib
fig = plt.figure(figsize=(10,6))
for i, im in enumerate(imagelist):
    fig.add_subplot(4,10,i+1).set_title('{}\n{}'.format(int(pred[i]),int(ans[i])))
    plt.axis('off')
    plt.imshow(im)
plt.show()